In [1]:
import pandas as pd
import numpy as np
import random
import torch
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650 Ti'

In [4]:
train = pd.read_csv('pca_train.csv')
test = pd.read_csv('pca_test.csv')
submit = pd.read_csv('sample_submission.csv')
train_y =pd.read_csv('train_label.csv')

In [5]:
train

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-6.227718,-2.070960,0.674094,-0.742135,0.309149,-4.006772,-0.664624,0.533987,0.024594,-0.195856,...,-2.114192,-0.605770,0.491399,-0.166693,1.289197,-0.403863,0.238731,2.166737,-0.106487,1.486114
1,-6.081077,-1.960213,0.846482,-0.932816,-0.050208,-4.082826,-0.449032,0.647842,0.417809,0.073842,...,-1.000466,-1.175968,0.299093,-1.187048,-0.499101,0.738016,0.632842,0.850430,-2.211427,1.083040
2,-6.394956,-2.954897,1.600033,-0.479463,-1.217065,-3.986472,-0.812934,0.472509,1.536917,1.989765,...,-1.703212,0.713673,0.330699,0.036953,-0.204944,0.930597,-0.693151,-0.572443,0.835598,-0.751483
3,-6.681355,-2.055031,1.144864,-0.517872,-0.168057,-3.929032,-0.718259,0.555827,0.096472,0.724261,...,0.434870,1.571784,-1.266089,1.584173,-0.290301,0.068907,-1.148258,1.904273,0.322937,-0.318100
4,-5.716713,-2.163729,0.877930,-0.605268,0.082372,-4.261466,-0.679734,0.739369,0.490033,0.546437,...,0.626112,-0.435030,2.112485,1.101894,1.746205,-0.445612,-1.518138,0.299177,-0.020655,0.234824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27666,-6.047007,-2.072704,0.668214,-0.254932,0.099894,-1.477478,-0.703836,-0.327026,0.504125,0.222820,...,-1.449840,-0.050902,-1.240334,-0.750522,-1.168852,0.048051,0.432534,-0.003478,-0.609472,-0.348196
27667,-6.569392,-1.932956,0.602381,0.253941,-0.341952,4.064471,-1.502157,-2.467482,0.281797,0.812378,...,0.721433,1.197452,-0.700384,-3.483315,-0.371596,0.478422,-1.537939,2.030344,-1.080110,-0.409972
27668,-6.232087,-1.834862,2.097659,-0.566552,-0.288520,-4.034954,0.457435,0.315611,0.981373,0.483152,...,0.662459,-1.293272,0.067110,0.357502,-0.155424,-0.658622,2.016063,-0.359339,0.993638,0.130081
27669,-6.387326,-0.387639,-1.090173,1.499431,0.629389,31.163396,-3.186039,-12.114586,-0.543894,-1.469931,...,-0.838120,0.584897,-1.355164,-1.358166,1.344522,-0.526878,-0.230260,-0.459776,2.507944,-1.379322


In [6]:
train_x_tensor = torch.FloatTensor(train.values).to(device)
train_y_tensor = torch.LongTensor(train_y.label).to(device)

test_x_tensor = torch.FloatTensor(test.values).to(device)

In [7]:
class NN(torch.nn.Module):
    def __init__(self):
        super(NN,self).__init__()
        self.linear1 = torch.nn.Linear(256,198, bias = True)
        self.linear2 = torch.nn.Linear(198,198, bias = True)
        self.linear3 = torch.nn.Linear(198, 198, bias = True)
        self.relu = torch.nn.ReLU()
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.relu(out)
        out = self.linear3(out)
        return out

model = NN().to(device)

In [8]:
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [9]:
model.train()
total_epoch = 50

for epoch in range(1, total_epoch + 1):
    
    optimizer.zero_grad()
    hypothesis = model(train_x_tensor)
    cost = loss(hypothesis, train_y_tensor)
    cost.backward()
        
    optimizer.step()
    
    if epoch % 100 ==0:
        print(epoch, cost.item())

In [10]:
model.eval()
with torch.no_grad():
    prediction = model(test_x_tensor)
    train_hypothesis = model(train_x_tensor)

In [11]:
from sklearn.metrics import accuracy_score
train_predict = torch.argmax(train_hypothesis, dim =1)
print(accuracy_score(train_predict.detach().cpu(), train_y.label))

0.8855841856094828


In [12]:
predict = torch.argmax(prediction, dim = 1)
submit.label = predict.detach().cpu().numpy()

In [13]:
submit

,id,label
0,0,113
1,1,170
2,2,68
3,3,28
4,4,121
...,...,...
13624,13624,166
13625,13625,69
13626,13626,23
13627,13627,68


In [14]:
submit.to_csv('2021_10_11_02.csv',index=False)

In [ ]:
class NN(torch.nn.Module):
    def __intit(self):
        super(NN,self).__init__()
        linear
        
    def forward(self,x):
        return out
    
nmodel = NN().to(device)